In [1]:
import os
import openai
import sys

In [2]:
openai.api_key  = os.environ['OPENAI_API_KEY']

In [3]:
!pip install pypdf chromadb tiktoken

  Obtaining dependency information for pypdf from https://files.pythonhosted.org/packages/1e/37/ba55cd56081eb8a606434332129f6a78cd52bdcfbb9bfcfa828147c9602c/pypdf-3.15.0-py3-none-any.whl.metadata
  Using cached tiktoken-0.4.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.3/270.3 kB 2.2 MB/s eta 0:00:00a 0:00:01


In [4]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    PyPDFLoader("https://maup.com.ua/assets/files/lib/book/p08_13.pdf"), #економіка підприємства#
    PyPDFLoader("http://eprints.kname.edu.ua/40297/1/2013%2096%D0%9B%20%D0%BF%D0%B5%D1%87%20%D0%9A%D0%9B%20%D0%A5%D0%9D%D0%A3%D0%9C%D0%93.pdf"), #ЕКОНОМІКА І ОРГАНІЗАЦІЯ ВИРОБНИЦТВА
    PyPDFLoader("Konspekt.pdf"), #Основи галузевої економіка та підприємництво
    PyPDFLoader("https://repository.kpi.kharkov.ua/server/api/core/bitstreams/c40886c5-fcad-4309-b74f-fa350ab11a94/content"), #МІЖНАРОДНА ЕКОНОМІКА: основні поняття
    PyPDFLoader("http://dspace.wunu.edu.ua/bitstream/316497/46548/1/%D0%9E%D0%BF%D0%BE%D1%80%D0%BD%D0%B8%D0%B9%20%D0%BA%D0%BE%D0%BD%D1%81%D0%BF%D0%B5%D0%BA%D1%82%20%D0%BB%D0%B5%D0%BA%D1%86%D1%96%D0%B9_%D0%9C%D0%95.pdf") #МІЖНАРОДНА ЕКОНОМІКА
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [5]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    keep_separator=False,
    chunk_size = 1500,
    chunk_overlap = 150
)

In [6]:
splits = text_splitter.split_documents(docs)

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [8]:
from langchain.vectorstores import Chroma

In [9]:
persist_directory = 'docs/chroma/'

In [10]:
#!rm -rf ./docs/chroma  # remove old database files if any

In [11]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [12]:
print(vectordb._collection.count())

2088


In [13]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Уяви себе студентом котрий здає тести на екзамені. Використовуй наступний кусок текста щоб дати відповіді на питання в тестах. 
Якщо не знаєш відоповіді, кажи шо не знаєш, не треба придумувати відповідь. Питання можуть включати варіанти відповідей. 
Тоді треба вибрати варіант і написати чому цей варіант підходить.

Формат відповіді:

%Варіат відповіді%

%Пояснення%

{context}
Питання: {question}
Відповідь:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [14]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name='gpt-3.5-turbo-0301', temperature=0)

In [15]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [177]:
#http://cit.vnu.edu.ua/wp-content/uploads/bases/applicant/applicant_TEST_438.html

question = """  
Оплата праці, за якою заробітна плата нараховується за встановленою тарифною ставкою чи окладом за фактично відпрацьований працівником час, – це:

відрядно-прогресивна
відрядна
почасова
відрядно-преміальна
"""


In [178]:
result = qa_chain({"query": question})

In [179]:
result["result"]

'почасова\n\nПояснення: За простої погодинної форми оплати праці заробіток працівника розраховується за формулою Zп.п = st, де s — годин на тарифна ставка відповідного розряду; t — кількість фактично відпрацьованих годин. Це означає, що заробітна плата нараховується за фактично відпрацьований час, тобто за принципом почасової оплати праці.'